<a href="https://colab.research.google.com/github/mvince33/Coding-Dojo/blob/main/week10/6_20_PCA_MotorData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predict Machine Failure

The task is to predict the condition of a motor based on the features provided.

This is ultimately a supervised learning task, but we will add PCA to our pre-processing step.

The goal here is to get the highest overall accuracy.

In [ ]:
import pandas as pd
from seaborn import heatmap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
# load data
df = pd.read_csv('https://raw.githubusercontent.com/ninja-josh/image-storage/da58a7aa38dccd3ea96dc2e8eac658d99d928625/Sensorless_RAW%20(1).csv')
df.head()

In [ ]:
# Check for duplicates
df.duplicated().sum()

In [ ]:
# Explore shape and datatypes of the data
df.info()


In [ ]:
# Check for missing values
df.isna().sum()

In [ ]:
# Explore the class balance in the target column.
# The last column (Column 49) is the target and is an integer (This is a classification task)
df['Column49'].value_counts()

In [ ]:
# Separate the target and features.  
X = df.drop(columns='Column49')
y = df['Column49']


In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X_train.describe()

In [ ]:
# Prepare preprocessing pipeline (What kinds of processing does this data need?)
# Remember, you MUST scale and center the data for use in PCA (it is distance based)




In [ ]:
# Create a PCA object that will retain 95% of the variance when transforming

# Check shape of new features after applying PCA


# Create a KNN classification model **WITHOUT** PCA and check the accuracy


In [ ]:
%%time
# Create and fit a KNN model WITHOUT PCA.


In [ ]:
%%time
# Make predictions and evaluate the model using classification_report


# Create a KNN classification model **WITH** PCA and check the accuracy

In [ ]:
%%time
# Create and fit a KNN model WITH PCA.


In [ ]:
%%time
# Make predictions and evaluate the model using classification_report


# Discuss:

1. What were the benefits of using PCA for this dataset?

2. What were the drawbacks?

3. Which model took longer to train?  
  
  - Which took longer to predict?  
  
  - Why might this be?  
  
  - What does this tell you about using PCA in a pipeline?

## Bonus! 

This is just a default model, but you could try a gridsearch or other classifier algorithms to reduce overfitting and see if we could get a better result!